In [ ]:
!pip install wikipedia-api
!pip install rdflib
!pip install openai
!pip install Wikidata
!pip install spacy wikidataintegrator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.0/149.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.1/102.1 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for shexer: filename=shexer-2.2.0-py3-none-any.whl size=156284 sha256=8a3f1beb7bafea2c1e73f4b51608e2ed37a621fe45e7ea6f1cd8883d4dbe8ebb
  Stored in directory: /root/.cache/pip/wheels/bd/1c/3b/402158f9715df4e67d384935f146eeb2eabe78f946c399b298
  Created whee

In [ ]:
import wikipediaapi
import nltk
from nltk.corpus import stopwords, wordnet
import string
from nltk.stem.wordnet import WordNetLemmatizer
import re
import spacy
from spacy.matcher import Matcher
import json
import requests
import spacy
from wikidataintegrator import wdi_core
import openai


# Download WordNet data (only need to do this once)
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Load SpaCy model and create a Matcher
nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)

In [ ]:
def json_dump(data, output_file_path):

  # Dump the data into a JSON file
  with open(output_file_path, "w") as output_file:
      json.dump(data, output_file, indent=4)

  print(f"Data has been dumped into '{output_file_path}'.")

In [ ]:
def get_wikidata_id_from_wikipedia(title):
    wikipedia_api_url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "format": "json",
        "prop": "pageprops",
        "titles": title,
        "formatversion": "2",
        "ppprop": "wikibase_item"
    }

    response = requests.get(wikipedia_api_url, params=params)
    data = response.json()

    wikidata_id = None
    if "query" in data and "pages" in data["query"]:
        page = data["query"]["pages"][0]
        if "pageprops" in page and "wikibase_item" in page["pageprops"]:
            wikidata_id = page["pageprops"]["wikibase_item"]

    return wikidata_id

In [ ]:
# URL of the Wikipedia API endpoint
api_url = "https://en.wikipedia.org/w/api.php"

# Parameters for the API request
params = {
    "action": "query",
    "format": "json",
    "titles": "Pacta_conventa_(Croatia)", #Donation_of_Constantine, #The_Protocols_of_the_Elders_of_Zion, #Salvator_Mundi_(Leonardo)
    "prop": "extracts",
    "explaintext": True,  # Get plain text instead of HTML
}

# Make the API request
response = requests.get(api_url, params=params)
data = response.json()

# Extract the plain text content
page = next(iter(data["query"]["pages"].values()))
plain_text = page["extract"]

temp = plain_text.split('== Footnotes ==')

plain_text = temp[0]

Pacta conventa (Latin for "agreed accords") was an agreement concluded between King Coloman of Hungary and the Croatian nobility in 1102 or afterwards, defining the status of Croatia in the union with Hungary. The earliest manuscript of the document is of the fourteenth century, so some historians believe it is likely a forgery.
The document titled Pacta conventa or Qualiter (the first word in the document) was found in a Trogir library. Until the 19th century it was considered that it dated to 1102. However, historians today hold that it is not an authentic document from 1102 and likely a forgery from the 14th century, but that the contents of the Pacta Conventa still correspond to the political situation of that time in Croatia. The document is preserved in the Hungarian National Museum in Budapest.


== Background ==
After Petar Snačić, the last Croatian king of Croat descent, was killed on the battlefield in 1097, the Croats had refused to surrender. To end the war, an agreement wa

In [ ]:
people = {}
potential_scholars = []

# Iterate through sentences
for sentence in plain_text.split("."):
    sentence = sentence.strip()
    if sentence:
        doc = nlp(sentence)
        # Extract scholars' names
        for ent in doc.ents:
            if ent.label_ == "PERSON":
                potential_scholars.append(ent.text)

print(potential_scholars)

# Query Wikidata and reconcile persons as instances of human
for person_name in list(set(potential_scholars)):
  query = f'SELECT ?item WHERE {{ ?item ?label "{person_name}"; wdt:P31 wd:Q5.}}'
  results = wdi_core.WDItemEngine.execute_sparql_query(query)

  if results["results"]["bindings"]:
      wikidata_id = results["results"]["bindings"][0]["item"]["value"].split("/")[-1]
      print(f"Name: {person_name}, Wikidata ID: {wikidata_id}")
      people.update({wikidata_id:[person_name]})
  else:
      api_url = f"https://www.wikidata.org/w/api.php?action=wbsearchentities&format=json&language={language}&search={person_name}"

      response = requests.get(api_url)
      data = response.json()

      # Extract the Wikidata ID from the first search result, if available
      if data.get("search"):
          wikidata_id = data["search"][0]["id"]
          query = f'SELECT ?item WHERE {{ wd:{wikidata_id} wdt:P31 wd:Q5.}}'
          results = wdi_core.WDItemEngine.execute_sparql_query(query)

          if results["results"]["bindings"]:
              print(f"Name: {person_name}, Wikidata ID: {wikidata_id}")
              people.update({wikidata_id:[person_name]})
      else:
          print("No matching entity found.")

['Qualiter', 'Petar Snačić', 'Kačić', 'Kukar', 'Thomas the Archdeacon', 'János Karácsonyi', 'Ljudmil Hauptmann', 'Miho Barada', 'Nada Klaić', 'Stjepan Antoljak', "Oleg Mandić's", 'Ferdo Šišić', 'Gyula Kristó', 'Bálint Hóman', 'János Karácsonyi', 'Drava', 'Sava', 'Croat', 'Coloman', 'Frederick Bernard Singleton', 'Croatia', 'Daniel Power', 'Croatia', 'Coloman', 'Nada Klaić', 'Matjaž Klemenčič', 'Žagar', 'Croatia', 'Louis II', 'Croatia', 'Nagodba', 'Croatia', 'Nagodba']
Name: Coloman, Wikidata ID: Q83066
Name: Matjaž Klemenčič, Wikidata ID: Q18644616
Name: Thomas the Archdeacon, Wikidata ID: Q1280353
Name: Stjepan Antoljak, Wikidata ID: Q12295365
Name: Daniel Power, Wikidata ID: Q68975620
Name: János Karácsonyi, Wikidata ID: Q1105077
Name: Ljudmil Hauptmann, Wikidata ID: Q12795030
Name: Petar Snačić, Wikidata ID: Q741354
Name: Nada Klaić, Wikidata ID: Q1276244
Name: Sava, Wikidata ID: Q47125681
Name: Louis II, Wikidata ID: Q316543
Name: Bálint Hóman, Wikidata ID: Q718762
No matching enti

In [ ]:
# add variants

for wiki_id, pers_label in people.items():
    query = f'SELECT ?nameLabel ?surnameLabel WHERE {{ wd:{wiki_id} wdt:P735 ?name; wdt:P734 ?surname. ?surname rdfs:label ?surnameLabel. ?name rdfs:label ?nameLabel}}'
    results = wdi_core.WDItemEngine.execute_sparql_query(query)

    if results["results"]["bindings"]:
        surname = results["results"]["bindings"][0]["surnameLabel"]["value"]
        name = results["results"]["bindings"][0]["nameLabel"]["value"]
        pers_label.append(name)
        pers_label.append(surname)

In [ ]:
print(people)
print(len(people))

{'Q83066': ['Coloman'], 'Q18644616': ['Matjaž Klemenčič', 'Matjaž', 'Klemenčič'], 'Q1280353': ['Thomas the Archdeacon'], 'Q12295365': ['Stjepan Antoljak'], 'Q68975620': ['Daniel Power', 'Daniel', 'Power'], 'Q1105077': ['János Karácsonyi', 'János', 'Karácsonyi'], 'Q12795030': ['Ljudmil Hauptmann'], 'Q741354': ['Petar Snačić'], 'Q1276244': ['Nada Klaić'], 'Q47125681': ['Sava', '罗伯托', 'Sava'], 'Q316543': ['Louis II', 'Louis', 'Grimaldi'], 'Q718762': ['Bálint Hóman'], 'Q1121105': ['Gyula Kristó'], 'Q1558008': ['Ferdo Šišić', 'Ferdo', 'Šišić'], 'Q16114446': ['Miho Barada', 'Miho', 'Barada'], 'Q1829652': ['Croat', 'Bernard', 'Croat'], 'Q59627892': ['Frederick Bernard Singleton', '弗雷德里克', 'Singleton']}
17


In [ ]:
def find_missing_elements(lst):
    missing_elements = []
    for i in range(len(lst) - 1):
        diff = lst[i + 1] - lst[i]
        if diff == 2:
            missing_elements.extend(range(lst[i] + 1, lst[i + 1]))
            lst.extend(missing_elements)
    return list(set(lst))

doc = nlp(plain_text)
sentences = list(doc.sents)

author_text_cluster = []
for wiki_id, pers_labels in people.items():

  temp = []
  for i, sentence in enumerate(sentences):
    for variant in pers_labels:
      if variant in str(sentence):
        temp.append(i)

  temp = list(set(temp))

  # Trova gli elementi mancanti
  temp = find_missing_elements(temp)

  # salva le frasi utili
  cluster_sentence = ''

  for i,sentence in enumerate(sentences):
    for i2 in temp:
      if i == i2:
        temp_sent = ''.join(str(sentence))
        cluster_sentence = cluster_sentence + temp_sent

  temp = (wiki_id, pers_labels, cluster_sentence)
  author_text_cluster.append(temp)

print(author_text_cluster)

[('Q83066', ['Coloman'], 'Pacta conventa (Latin for "agreed accords") was an agreement concluded between King Coloman of Hungary and the Croatian nobility in 1102 or afterwards, defining the status of Croatia in the union with Hungary.The Croatian nobles allegedly concluded the Pacta conventa with King Coloman before his crowning as the Croatian king in Biograd.The Hungarian king offered "an agreement as pleases them" to the twelve noble Croatian tribes from the families of Čudomirić, Gusić, Kačić, Kukar, Jamomet, Lasničić, Lapčan and Karinjan, Mogorović, Poletčić, Snačić, Šubić, and Tugomirić.\n\n\n== Terms ==\nThe agreement determined that the Croatian nobles who signed the document with King Coloman would retain their possessions and properties without interference.The Croatian nobility retained its laws and privileges including the restriction of military service that they owed to the king within the boundaries of Croatia.\n\n\n== Interpretations of the agreement ==\nAccording to t

In [ ]:
def question_prompt(prompt, searched_category_string, n_max_tokens):

  # Use the OpenAI API to generate the desired output
  response = openai.Completion.create(
      engine="text-davinci-003",
      prompt=prompt,
      max_tokens=n_max_tokens
  )

  output_text = response.choices[0].text.strip()
  #claim_author= output_text.split("\nAuthenticity assessment (forgery or authentic):")

  print(searched_category_string, output_text)
  return output_text

In [ ]:
# Set your OpenAI API key
api_key = ""

# Initialize the OpenAI library with your API key
openai.api_key = api_key

for cluster in author_text_cluster:

  print('/n')
  print(cluster[1][0])

  assessment_prompt = f"\nText: {cluster[2]}\n \n Guess if {cluster[1][0]} opinion is assessing the artwork as authentic or a forgery. Answer with authentic vs forgery and your degree of certainty."
  assessment_res = question_prompt(assessment_prompt, 'Artwork authenticity assessment:', 10)

  original_date_prompt = f"\nText: {cluster[2]}\n \n, If available in the input text,{cluster[1][0]} extract which is the date deemed to be the creation date of the artwork by {cluster[1][0]}. Answer with \Original date:\""
  original_date_res = question_prompt(original_date_prompt, 'Original date of creation:', 10)

  study_type_prompt = f"\nText: {cluster[2]}\n \n, If available in the input text, which method used the {cluster[1][0]} to asses the document. Answer with \Study Type\""
  study_type_res = question_prompt(study_type_prompt, 'Field of study:', 10)

  feature_analysis_prompt = f"\nText: {cluster[2]}\n \n, If available in the input text, which are the features of the text studied by the {cluster[1][0]} to assess its authenticity. Answer with \Analysis:\ and report only the terminology used in the provided text"
  feature_analysis_res = question_prompt(feature_analysis_prompt, 'Feature analysis:', 50)



/n
Coloman
Artwork authenticity assessment: Authentic - 100% Certain
Original date of creation: 1102 or afterwards"
Field of study: Study Type: Country Study
Feature analysis: Analysis: Pacta Conventa, King Coloman, Croatian nobility, Croatian tribes, Croatian nobility's laws and privileges, military service, Ladislaus I of Hungary, Croatian and Hungarian crowns, maritime cities.
/n
Matjaž Klemenčič
Artwork authenticity assessment: Forgery, 100%.
Original date of creation: 1102"
Field of study: Historical Analysis
Feature analysis: Analysis: The Pacta Conventa, Croatian position, Croatian nobility, Habsburg monarchy.
/n
Thomas the Archdeacon
Artwork authenticity assessment: Forgery, 100% certainty.
Original date of creation: Not available"
Field of study: Study Type: Historical Research
Feature analysis: .

Analysis: Document authenticity, 14th-century "addendum".
/n
Stjepan Antoljak
Artwork authenticity assessment: Authentic - Certain
Original date of creation: ". 

Original date: ""


KeyboardInterrupt: ignored